# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, see [this forum post](https://carnd-forums.udacity.com/cq/viewquestion.action?spaceKey=CAR&id=29496372&questionTitle=finding-lanes---import-cv2-fails-even-though-python-in-the-terminal-window-has-no-problem-with-import-cv2) for more troubleshooting tips.**  

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
#for linear regression support
from scipy import stats
%matplotlib inline

## Read in an Image

In [2]:
#reading in an image
image = mpimg.imread('test_images_output/output_solidYellowLeft.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)


## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [257]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines_old(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    
    # The method below is fast but more vulnerable to noises
    # initialize the top and bottom of each lane
    imshape = img.shape
    top_left = [imshape[1],imshape[0]]
    bottom_left = [imshape[1],0]
    top_right = [0,imshape[0]]
    bottom_right = [0,0]
    # fine the true top and bottom of each line
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = ((y2-y1)/(x2-x1))
            if slope < 0:
                if min(x1, x2) <= bottom_left[0]:
                    bottom_left[0] = min(x1, x2)
                    bottom_left[1] = max(y1, y2)
                if min(y1, y2) <= top_left[1]:
                    top_left[1] = min (y1, y2)
                    top_left[0] = max (x1, x2)
            else:
                if max(x1, x2) >= bottom_right[0]:
                    bottom_right[0] = max(x1, x2)
                    bottom_right[1] = max(y1, y2)
                if min(y1, y2) <= top_right[1]:
                    top_right[1] = min(y1, y2)
                    top_right[0] = min(x1, x2) 
    # Calculate slope
    left_slope = (top_left[1]-bottom_left[1])/(top_left[0]-bottom_left[0])
    right_slope = (top_right[1]-bottom_right[1])/(top_right[0]-bottom_right[0])
    # predict new bottom of each line 
    predict_bottom_left_x = int((imshape[0]-top_left[1])/left_slope + top_left[0])
    predict_bottom_right_x = int((imshape[0]-top_right[1])/right_slope + top_right[0])
    # predict new top of each line by absolute position
    mask_top = imshape[1]/2-50
    predict_top_left_x = int((mask_top-bottom_left[1])/left_slope + bottom_left[0])
    predict_top_right_x = int((mask_top-bottom_right[1])/right_slope + bottom_right[0])
    # draw the lines
    cv2.line(img, (top_left[0], top_left[1]), (predict_bottom_left_x, imshape[0]), color, thickness)
    cv2.line(img, (top_right[0], top_right[1]), (predict_bottom_right_x, imshape[0]), color, thickness)

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    # create list to store x, y values as vector X, Y for each line
    left_line_x = []
    left_line_y = []
    right_line_x = []
    right_line_y = []
    # get image shape to accomodate different resolution
    imshape = img.shape
    y_top = int(0.63*imshape[0])
    y_bottom = imshape[0]
    # groupe the coordinates for two lines
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = ((y2-y1)/(x2-x1))
            if slope <= 0:
                left_line_x += [x1, x2]
                left_line_y += [y1, y2]
            else:
                right_line_x += [x1, x2]
                right_line_y += [y1, y2] 
    # make sure when hough-transform fails to detect any line, the draw-line will not crash
    # use linear regression to extrapolate lines
    if len(left_line_x) and len(left_line_y):
        k_left, b_left, r_value, p_value, std_err = stats.linregress(left_line_x,left_line_y)
        x_top_left = int((y_top-b_left)/(k_left))
        x_bottom_left = int((y_bottom-b_left)/(k_left))
        cv2.line(img, (x_top_left, y_top), (x_bottom_left, y_bottom), color, thickness)
    if len(right_line_x) and len(right_line_y):
        k_right, b_right, r_value, p_value, std_err = stats.linregress(right_line_x,right_line_y)
        x_top_right = int((y_top-b_right)/(k_right))
        x_bottom_right = int((y_bottom-b_right)/(k_right))
        cv2.line(img, (x_top_right, y_top), (x_bottom_right, y_bottom), color, thickness)
    
    
    
def hough_lines(img, rho = 1, theta = np.pi/180, threshold = 80, min_line_len = 10, max_line_gap = 2):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img


# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [258]:
import os
os.listdir("test_images/")

['solidWhiteCurve.jpg',
 'solidYellowCurve2.jpg',
 'whiteCarLaneSwitch.jpg',
 'solidYellowCurve.jpg',
 'solidYellowLeft.jpg',
 'solidWhiteRight.jpg']

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [259]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.
def lane_finding_pipeline(img):
    imshape = img.shape
    vertices = np.array([[(100,540),(380,350), (580,350), (900,540)]], dtype=np.int32)
    gray_img = grayscale(img)
    canny_img = canny(gray_img, 50, 150)
    blur_img = gaussian_blur(canny_img,5)
    masked_img = region_of_interest(blur_img, vertices)
    #plt.imshow(masked_img)
    line_img = hough_lines(masked_img, 1, np.pi/180, 100, 40, 30)
    return weighted_img(line_img, img)

# setup output path
out_path = "test_images_output/"

img_paths = os.listdir("test_images/")
for img_path in img_paths:
    image = mpimg.imread("test_images/"+img_path)
    result_image = lane_finding_pipeline(image)
    cv2.imwrite(os.path.join(out_path, "output_"+img_path), cv2.cvtColor(result_image, cv2.COLOR_RGB2BGR))
    #plt.imshow(result_image)

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an `import error` when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt. Also, check out [this forum post](https://carnd-forums.udacity.com/questions/22677062/answers/22677109) for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://carnd-forums.udacity.com/display/CAR/questions/26218840/import-videofileclip-error) for more troubleshooting tips across operating systems.**

In [260]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [261]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    result = lane_finding_pipeline(image)

    return result

Let's try the one with the solid white lane on the right first ...

In [262]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:06<00:00, 38.50it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 5.4 s, sys: 756 ms, total: 6.15 s
Wall time: 6.85 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [264]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [265]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:23<00:00, 29.30it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 18.4 s, sys: 2.1 s, total: 20.5 s
Wall time: 23.8 s


In [266]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [267]:
def lane_finding_pipeline_2masks(img):
    imshape = img.shape
    # these absolute vertices only work for 960*540
    # create mask for left lane
    vertices_1 = np.array([[(200,500),(400,350), (450,350), (300,500)]], dtype=np.int32)
    # create mask for right lane
    vertices_2 = np.array([[(750,500),(500,350), (600,350), (800,500)]], dtype=np.int32)
    gray_img = grayscale(img)
    #plt.imshow(gray_img)
    canny_img = canny(gray_img, 50, 120)
    blur_img = gaussian_blur(canny_img,5)
    masked_img_left = region_of_interest(blur_img, vertices_1)
    masked_img_right = region_of_interest(blur_img, vertices_2)
    masked_img = weighted_img(masked_img_left, masked_img_right, 1, 1, 0)
    #plt.imshow(masked_img)
    line_img = hough_lines(masked_img, 1, np.pi/180, 100, 30, 40)
    return weighted_img(line_img, img)

# pictures are down-graded to 960*540
img_paths = os.listdir("challenge_images/")
for img_path in img_paths:
    image = mpimg.imread("challenge_images/"+img_path)
    result_image = lane_finding_pipeline_2masks(image)
    cv2.imwrite(os.path.join(out_path, "output_two_mask_"+img_path), cv2.cvtColor(result_image, cv2.COLOR_RGB2BGR))

In [272]:
def color_trans(img):
    """Applies the V transform"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

def lane_finding_pipeline_robust(img):
    imshape = img.shape
    # need to use relative vertices because it's 1280*720 this time
    vertices = np.array([[(int(0.2*imshape[1]),int(0.88*imshape[0])),(int(0.41*imshape[1]),int(0.65*imshape[0])), (int(0.63*imshape[1]),int(0.65*imshape[0])), (int(0.81*imshape[1]),int(0.88*imshape[0]))]], dtype=np.int32)
    color_trans_img = color_trans(img)
    canny_img = canny(color_trans_img, 150, 250)
    #plt.imshow(canny_img)
    blur_img = gaussian_blur(canny_img,5)
    masked_img = region_of_interest(blur_img, vertices)
    #plt.imshow(masked_img)
    line_img = hough_lines(masked_img, 1, np.pi/180, 100, 40, 20)
    #plt.imshow(line_img)
    return weighted_img(line_img, img)

img_paths = os.listdir("challenge_images/")
for img_path in img_paths:
    image = mpimg.imread("challenge_images/"+img_path)
    result_image = lane_finding_pipeline_robust(image)
    cv2.imwrite(os.path.join(out_path, "output_color_trans_"+img_path), cv2.cvtColor(result_image, cv2.COLOR_RGB2BGR))

In [273]:
def process_image_robust(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    #result = lane_finding_pipeline_2masks(image)
    result = lane_finding_pipeline_robust(image)
    return result

In [274]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image_robust)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|██████████| 251/251 [00:25<00:00,  9.77it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

CPU times: user 23.9 s, sys: 1.51 s, total: 25.5 s
Wall time: 27.9 s


In [275]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))